In [83]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [108]:
import numpy as np
import pandas as pd
import datetime as dt
import dateutil.relativedelta as rd

# Reflect Tables into SQLAlchemy ORM

In [85]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
#census_data = pd.read_sql("SELECT * FROM Census_Data", conn)
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
for Class in Base.classes.keys():
    print(Class)

measurement
station


In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [9]:
# some classes to display queries
def print_query(query):
    """
    Display all the variables in the item query.
    
        query -- A query, which in this case could be a class instance or a collection of class instances as a sqlalchemy.orm.query.Query.
    """
    if type(query) is sqlalchemy.orm.query.Query:
        for item in query:
            print_query(item)
    else:
        print(f"{'-'*4}class--{query.__class__.__name__}{'-'*4}")
        for key, value in query.__dict__.items():
            if not key.startswith("_"):
                print(f'{key} : {value}')

In [10]:
# Display some starter data
print_query(session.query(Measurement).first())
print_query(session.query(Station).limit(1))

----class--measurement----
id : 1
prcp : 0.08
station : USC00519397
tobs : 65.0
date : 2010-01-01
----class--station----
longitude : -157.8168
station : USC00519397
id : 1
elevation : 3.0
latitude : 21.2716
name : WAIKIKI 717.2, HI US


# Exploratory Precipitation Analysis

In [153]:
datetime_format = "%Y-%m-%d"

def to_datetime(value):
    """
    Convert the string into a datetime in a more compact function
    """
    # format using this template
    return dt.datetime.strptime(value, datetime_format)

# Find the most recent date in the data set.
most_recent_date = to_datetime(
    # Query the database on the class Measurement
    session.query(Measurement).\
        # order the column, use the decending argument
        order_by(Measurement.date.desc()).\
        # grab only the date column
        first().date)

most_recent_date

datetime.datetime(2017, 8, 23, 0, 0)

In [160]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = most_recent_date - rd.relativedelta(years = 1)

# Perform a query to retrieve the data and precipitation scores
precipitation_scores = session.query(Measurement).\
    filter(
    Measurement.date >= one_year_ago.date(), 
    Measurement.date <= most_recent_date.date()
)
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_scores_df = pd.read_sql(precipitation_scores.statement, session.bind)
precipitation_scores_df.set_index('id', inplace = True)
precipitation_scores_df['date'] = pd.to_datetime(
    precipitation_scores_df['date'], 
    format=datetime_format
)
# Sort the dataframe by date
precipitation_scores_df.sort_values(by = ['date'], inplace = True)

# Use Pandas Plotting with Matplotlib to plot the data


In [11]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number stations in the dataset


In [13]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [66]:
# Close Session
session.close()